<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:


      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-23 13:06:14
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: 28.68 K (0.2%)
Current PnL: -25.45 L (-16.44%)
CY Booked + Current PnL: -11.16 L (-7.21%)
-------------------
Total profit:  1.62 L
Total loss:  -27.08 L
-------------------
Total Booked + Current PnL: 15.82 L (12.37%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.09%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.09 L (63.62%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-1.22,-19.02,23.60,0.09,19261.0,-19164.0,81613.0,89.33,39.0,M-SC,2.39,253.0,-0.99,0.58,7.22,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.25,8.31,10.51,19.69,21827.0,15933.0,207675.0,-2.32,54.0,X-MC,2.48,78.0,0.73,1.47,15.67,XY25,NTT,AC
50,MASFIN,398.61,1.66,-1.65,24.10,22.05,23223.0,-1620.0,96360.0,-15.09,56.0,H-SC,6.36,164.0,-0.07,0.68,40.07,XR,ATH,FINANCE
43,ITC,452.00,0.52,-0.79,11.66,10.78,27801.0,-1891.0,238427.0,-40.74,53.0,X-LC,2.11,5.0,-0.07,1.69,4.41,X40,NTT,FMCG
52,NESTLEIND,1377.00,1.03,10.83,9.53,21.40,28036.0,28759.0,294185.0,2.90,56.0,X-LC,5.45,12.0,1.03,2.09,18.20,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TITAGARH,1548.00,3.86,-20.16,82.55,45.74,159605.0,-48833.0,193344.0,6.29,60.0,H-SC,6.29,174.0,-0.31,1.37,25.75,XY24,NTT,ENGINEERING
17,COALINDIA,484.83,3.66,-1.54,21.09,19.23,31244.0,-2312.0,148148.0,15.50,73.0,L-LC,2.13,179.0,-0.07,1.05,19.41,XY25,ATH,MINING
18,COFFEEDAY,80.00,3.58,-39.28,122.78,35.27,84653.0,-44602.0,68947.0,-53.72,52.0,L-SC,9.72,270.0,-0.53,0.49,68.75,XR,NTT,HOTELS
58,RECLTD,446.00,3.56,-3.48,25.09,20.74,49202.0,-7068.0,196102.0,26.46,57.0,M-MC,8.46,189.0,-0.14,1.39,6.77,XY25,NTT,FINANCE
79,VAIBHAVGBL,521.00,3.20,-25.20,114.93,60.77,157125.0,-46061.0,136714.0,57.58,53.0,H-SC,4.71,159.0,-0.29,0.97,23.12,XR,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,-4.94,-57.64,136.23,0.07,79162.0,-79068.0,58109.0,2030.05,67.0,L-SC,3.57,268.0,-1.00,0.41,44.52,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.0,-2.73,-4.85,90.94,81.67,81170.0,-4553.0,89257.0,8033.33,66.0,L-SC,12.11,271.0,-0.06,0.63,72.85,XR,NTT,CERAMICS
14,CAMPUS,393.0,-1.92,-11.86,51.21,33.28,75731.0,-19892.0,147883.0,-29.02,41.0,M-SC,1.88,222.0,-0.26,1.05,14.62,XY24,NTT,FOOTWEAR
83,WHIRLPOOL,2270.0,-1.88,-24.70,141.59,81.92,122395.0,-28355.0,86443.0,-55.95,32.0,M-SC,20.67,236.0,-0.23,0.61,3.12,XR,NTT,DURABLES
27,GLAXO,3466.2,-1.41,-0.13,39.32,39.15,78259.0,-252.0,199032.0,-19.83,38.0,X-MC,10.37,60.0,-0.00,1.41,28.84,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.75,-1.37,110.81,107.91,161358.0,-2027.0,145617.0,-19.74,57.0,M-SC,11.90,216.0,-0.01,1.03,8.58,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.76,-16.80,20.22,0.02,29365.0,-29330.0,145229.0,121.54,41.0,M-SC,12.02,234.0,-1.00,1.03,26.32,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.22,-19.02,23.60,0.09,19261.0,-19164.0,81613.0,89.33,39.0,M-SC,2.39,253.0,-0.99,0.58,7.22,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,-0.50,-21.73,68.68,32.03,70870.0,-28646.0,103189.0,-15.37,62.0,H-SC,9.10,148.0,-0.40,0.73,15.58,OX40N,NTT,IT
66,SIS,528.00,-0.72,-24.05,59.66,21.26,50310.0,-26704.0,84328.0,1989.64,50.0,H-SC,4.73,166.0,-0.53,0.60,13.94,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.63,11.14,55.0,72.27,103574.0,18869.0,188317.0,-4.72,73.0,M-LC,3.68,99.0,0.18,1.34,18.0,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.63,11.14,55.00,72.27,103574.0,18869.0,188317.0,-4.72,73.0,M-LC,3.68,99.0,0.18,1.34,18.00,XR,NTT,IT
25,FINCABLES,1641.55,-0.75,-1.37,110.81,107.91,161358.0,-2027.0,145617.0,-19.74,57.0,M-SC,11.90,216.0,-0.01,1.03,8.58,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.99,-3.74,114.76,106.72,136243.0,-4616.0,118720.0,-52.34,41.0,H-SC,4.29,139.0,-0.03,0.84,18.76,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.88,-2.32,38.85,35.63,79531.0,-4862.0,204712.0,-16.30,46.0,H-MC,2.39,119.0,-0.06,1.45,13.97,AR,ATH,PHARMA
6,ASIANTILES,137.00,-2.73,-4.85,90.94,81.67,81170.0,-4553.0,89257.0,8033.33,66.0,L-SC,12.11,271.0,-0.06,0.63,72.85,XR,NTT,CERAMICS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.01,-17.84,89.24,55.47,178369.0,-43410.0,199876.0,-27.18,32.0,X-MC,15.19,64.0,-0.24,1.42,2.41,X40N,ATH,FINANCE
8,AWL,485.00,-0.79,-24.11,102.25,53.50,234162.0,-72742.0,229009.0,-63.25,33.0,X-MC,5.40,58.0,-0.31,1.62,1.72,XY24,NTT,FMCG
22,DIXON,18931.72,-0.33,-14.74,47.87,26.07,79674.0,-28782.0,166439.0,-58.54,33.0,X-MC,11.86,56.0,-0.36,1.18,3.66,X40N,ATH,IT
34,HONAUT,58357.33,-0.57,-17.95,74.67,43.32,99789.0,-29232.0,133640.0,-29.54,33.0,X-SC,5.64,90.0,-0.29,0.95,2.61,X40N,ATH,ELECTRICAL
20,DABUR,735.00,-0.17,-3.64,49.13,43.70,117436.0,-9036.0,239032.0,-11.62,35.0,X-MC,2.08,72.0,-0.08,1.70,10.86,XY24,BTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,0.73,-2.71,24.52,21.14,54231.0,-6169.0,221170.0,-9.65,47.0,X-MC,1.18,74.0,-0.11,1.57,24.66,XY24,NTT,PAINTS
15,CAMS,950.0,-0.39,-0.52,25.13,24.48,62960.0,-1320.0,250536.0,-81.09,48.0,X-SC,1.18,86.0,-0.02,1.78,22.49,X40N,NTT,MISC
20,DABUR,735.0,-0.17,-3.64,49.13,43.70,117436.0,-9036.0,239032.0,-11.62,35.0,X-MC,2.08,72.0,-0.08,1.70,10.86,XY24,BTT,FMCG
43,ITC,452.0,0.52,-0.79,11.66,10.78,27801.0,-1891.0,238427.0,-40.74,53.0,X-LC,2.11,5.0,-0.07,1.69,4.41,X40,NTT,FMCG
78,UNITDSPR,1644.0,0.32,5.13,14.87,20.77,36605.0,12022.0,246166.0,-3.99,52.0,X-MC,2.11,62.0,0.33,1.75,12.41,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-1.06,-25.88,121.48,64.15,214242.0,-61591.0,176360.0,-56.75,38.0,X-SC,3.80,82.0,-0.29,1.25,0.69,XY24,BTT,CEMENT
19,COLPAL,3726.84,-0.26,-20.16,77.23,41.51,162399.0,-53085.0,210280.0,-4.98,39.0,X-MC,7.74,61.0,-0.33,1.49,0.74,XY25,ATH,FMCG
11,BATAINDIA,2096.00,0.04,-38.82,121.00,35.20,95248.0,-49953.0,78717.0,3.25,35.0,X-SC,16.87,92.0,-0.52,0.56,0.97,X40,NTT,FOOTWEAR
59,RELAXO,1176.00,0.15,-47.81,191.81,52.31,145323.0,-69396.0,75764.0,-44.44,42.0,X-SC,7.27,91.0,-0.48,0.54,1.41,X40N,NTT,FOOTWEAR
8,AWL,485.00,-0.79,-24.11,102.25,53.50,234162.0,-72742.0,229009.0,-63.25,33.0,X-MC,5.40,58.0,-0.31,1.62,1.72,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.12,-28.58,100.00,42.84,46428.0,-18578.0,46428.0,-53.17,49.0,X-SC,34.23,83.0,-0.40,0.33,5.04,XY24,NTT,MISC
59,RELAXO,1176.00,0.15,-47.81,191.81,52.31,145323.0,-69396.0,75764.0,-44.44,42.0,X-SC,7.27,91.0,-0.48,0.54,1.41,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.04,-38.82,121.00,35.20,95248.0,-49953.0,78717.0,3.25,35.0,X-SC,16.87,92.0,-0.52,0.56,0.97,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.12,-2.46,27.20,24.08,33681.0,-3121.0,123829.0,-10.30,42.0,X-SC,5.86,89.0,-0.09,0.88,16.68,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.57,-17.95,74.67,43.32,99789.0,-29232.0,133640.0,-29.54,33.0,X-SC,5.64,90.0,-0.29,0.95,2.61,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-0.58,-9.11,32.80,20.71,112760.0,-34455.0,343782.0,-21.68,68.0,X-LC,3.46,1.0,-0.31,2.44,14.87,X40,ATH,IT
41,INFY,1972.00,-1.39,13.42,18.36,34.24,65156.0,41991.0,354879.0,-11.70,68.0,X-LC,5.80,2.0,0.64,2.52,22.81,X40,NTT,IT
75,TMPV,600.00,1.35,-25.39,64.84,22.98,109276.0,-57366.0,168532.0,-23.08,55.0,X-LC,6.40,3.0,-0.52,1.20,6.00,XY24,NTT,AUTO
81,VBL,671.64,-0.84,-2.73,39.47,35.67,121263.0,-8613.0,307229.0,-14.36,57.0,X-LC,3.61,4.0,-0.07,2.18,10.75,X40N,ATH,FMCG
43,ITC,452.00,0.52,-0.79,11.66,10.78,27801.0,-1891.0,238427.0,-40.74,53.0,X-LC,2.11,5.0,-0.07,1.69,4.41,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.88,-37.33,112.04,32.89,54213.0,-28820.0,48387.0,-697.80,54.0,L-MC,6.80,259.0,-0.53,0.34,33.25,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-4.94,-57.64,136.23,0.07,79162.0,-79068.0,58109.0,2030.05,67.0,L-SC,3.57,268.0,-1.00,0.41,44.52,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-2.73,-4.85,90.94,81.67,81170.0,-4553.0,89257.0,8033.33,66.0,L-SC,12.11,271.0,-0.06,0.63,72.85,XR,NTT,CERAMICS
50,MASFIN,398.61,1.66,-1.65,24.10,22.05,23223.0,-1620.0,96360.0,-15.09,56.0,H-SC,6.36,164.0,-0.07,0.68,40.07,XR,ATH,FINANCE
13,BSOFT,831.70,1.52,-17.15,81.99,50.79,95547.0,-24118.0,116535.0,6.30,74.0,H-SC,7.65,151.0,-0.25,0.83,35.68,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.52,-17.15,81.99,50.79,95547.0,-24118.0,116535.0,6.30,74.0,H-SC,7.65,151.0,-0.25,0.83,35.68,XR,ATH,IT
56,RAJESHEXPO,518.00,-4.94,-57.64,136.23,0.07,79162.0,-79068.0,58109.0,2030.05,67.0,L-SC,3.57,268.0,-1.00,0.41,44.52,OX40N,NTT,JEWELLERY
61,ROUTE,2227.21,0.07,-45.59,210.72,69.05,152555.0,-60669.0,72397.0,-55.87,56.0,H-SC,20.98,140.0,-0.40,0.51,11.65,SR,ATH,IT
6,ASIANTILES,137.00,-2.73,-4.85,90.94,81.67,81170.0,-4553.0,89257.0,8033.33,66.0,L-SC,12.11,271.0,-0.06,0.63,72.85,XR,NTT,CERAMICS
84,WIPRO,420.00,-0.63,11.14,55.00,72.27,103574.0,18869.0,188317.0,-4.72,73.0,M-LC,3.68,99.0,0.18,1.34,18.00,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,44.81
2,50,76.54


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.91
MC    30.73
LC    24.32
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.78
X40      23.01
X40N     12.95
XR        9.93
XY25      9.11
AR        8.91
OX40N     7.52
SR        1.00
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.20
H-SC    23.82
X-LC    20.82
M-SC    11.57
X-SC     7.99
H-MC     4.80
L-SC     1.53
M-MC     1.39
M-LC     1.34
H-LC     1.11
L-LC     1.05
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.19,-7.65,42.27
IT,13.17,-13.27,70.22
FINANCE,9.56,-18.18,69.58
MISC,7.27,-26.72,78.56
ELECTRICAL,5.95,-11.88,52.86
PAINTS,5.78,-13.27,30.10
INSURANCE,4.75,-1.30,35.97
PHARMA,4.06,-2.51,34.78
AUTO,2.87,-29.87,74.90


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3147210.0,21
AR,1314965.0,10
XR,1262278.0,13
X40,979409.0,14
X40N,861567.0,9
OX40N,729865.0,10
XY25,355999.0,6
SR,278680.0,2
MH,78965.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3617756.0,25
M-SC,1427771.0,15
X-MC,1414779.0,16
X-SC,794550.0,8
X-LC,792216.0,11
H-MC,401585.0,3
L-SC,244985.0,3
M-LC,103574.0,1
H-LC,77063.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1260003.0      6
           AR         902930.0      5
M-SC       XY24       811228.0      6
H-SC       XR         765047.0      7
X-MC       X40        486327.0      7
           XY24       405829.0      3
           X40N       338397.0      4
H-SC       OX40N      332131.0      4
M-SC       OX40N      318572.0      5
X-SC       X40N       308072.0      3
X-LC       X40        300955.0      5
X-SC       XY24       294351.0      3
H-SC       SR         278680.0      2
X-LC       X40N       215098.0      2
H-MC       AR         210622.0      2
X-SC       X40        192127.0      2
H-MC       XY24       190963.0      1
X-LC       XY24       184836.0      2
X-MC       XY25       184226.0      2
M-SC       XR         173621.0      2
L-SC       XR         165823.0      2
M-SC       AR         124350.0      2
M-LC       XR         103574.0      1
X-LC       XY25        91327.0      2
L-SC       OX40N       79162.0      1
H-SC       MH          78965.0      1
H-LC       AR          77063.0      1
L-MC       XR          54213.0      1
M-MC       XY25        49202.0      1
L-LC       XY25        31244.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
